# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [ ]:
from sagemaker import get_execution_role

In [ ]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 4개의 권한 추가
    - AmazonBedrockFullAccess
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess
    - AWSCloud9SSMAccessRole

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

- #### **Please change to `install_complex_pdf = True` when utilizing Complex PDF**.
    - ####  [Note] install_complex_pdf will take about 30 minutes.

In [ ]:
install_needed = True
install_complex_pdf = False

In [ ]:
import os
import sys
import IPython
import subprocess

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain
    !{sys.executable} -m pip install -U langchain-community
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy #==2.0.1
    !{sys.executable} -m pip install -U pypdf
    #!{sys.executable} -m pip install -U spacy
    #!{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython
    !{sys.executable} -m pip install -U ipywidgets
    #!{sys.executable} -m pip install -U llmsherpa
    !{sys.executable} -m pip install -U anthropic
    !{sys.executable} -m pip install -U faiss-cpu
    !{sys.executable} -m pip install -U jq
    !{sys.executable} -m pip install -U pydantic

    if install_complex_pdf:
        
        response = subprocess.run(['cat', '/etc/os-release'], capture_output=True)
        response = response.stdout.decode("utf-8")
        
        if "Amazon Linux" in response: ## SageMaker Notebook

            !sudo rpm -Uvh https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
            !sudo yum -y update
            !sudo yum install -y poppler-utils
            !{sys.executable} -m pip install -U lxml
            !{sys.executable} -m pip install -U kaleido
            !{sys.executable} -m pip install -U uvicorn
            !{sys.executable} -m pip install -U pandas
            !{sys.executable} -m pip install -U numexpr
            !{sys.executable} -m pip install -U pdf2image
            !sudo sh install_tesseract.sh
            !sudo amazon-linux-extras install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !sudo rm -rf leptonica-1.84.1 leptonica-1.84.1.tar.gz tesseract-ocr

        else: ## SageMaker Studio
            !sudo apt-get install software-properties-common -y
            !sudo add-apt-repository ppa:alex-p/tesseract-ocr5 -y
            !sudo apt-get update -y
            !sudo apt-get install poppler-utils tesseract-ocr -y
            !sudo apt-get install libgl1-mesa-glx libglib2.0-0 -y
            !sudo apt install libreoffice -y
            !{sys.executable} -m pip install -U "unstructured[all-docs]"
            !{sys.executable} -m install -U pdf2image
            #!cd /usr/share/tesseract-ocr/5/tessdata/ && sudo wget https://github.com/tesseract-ocr/tessdata_best/raw/main/kor.traineddata

    IPython.Application.instance().kernel.do_shutdown(True)

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [ ]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [ ]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

In [ ]:
bedrock_region=boto3.Session().region_name

In [ ]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    endpoint_url=f"https://bedrock-runtime.{bedrock_region}.amazonaws.com"
)

In [ ]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

## 5. generation

In [ ]:
import base64
from langchain.llms.bedrock import Bedrock
from langchain_community.chat_models import BedrockChat
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

### Titan

In [ ]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [ ]:
print (colored(llm(prompt), "green"))

### Claude v2.1

In [ ]:
llm = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens": 1024,
        "temperature":0,
        "top_p":0.9,
        "stop_sequences": ["\n\nHuman"],
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm

In [ ]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to ask you a question.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

human_prompt = [
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

In [ ]:
response = chain.invoke(
    {
        "question": "세이지메이커의 장점을 100단어로 알려주세요."
    }
)

### Claude v3 sonnet and haiku
## test: 아래 이미지에 대한 질문
![nn](../imgs/setup/attention-is-all-you-needs.png)

In [ ]:
llm = BedrockChat(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Haiku"), # or "Claude-V3-Sonnet"
    client=bedrock_client,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
        # "temperature": 0,
        # "top_k": 350,
        # "top_p": 0.999
    }
)
llm

In [ ]:
system_prompt = '''
                You are a master answer bot designed to answer user's questions.
                I'm going to give you contexts which consist of texts and images.
                Read the contexts carefully, because I'm going to ask you a question about it.
                '''
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)

In [ ]:
with open("../imgs/setup/attention-is-all-you-needs.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
    base64_string = encoded_string.decode('utf-8')
    
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{base64_string}",
        },
    },
    {
        "type": "text",
        "text": '''
                Here is the question: <question>{question}</question>

                Answer in Korean.
                If the question cannot be answered by the contexts, say "No relevant contexts".
                
        '''
    }           
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [system_message_template, human_message_template]
)

chain = prompt | llm | StrOutputParser()

response = chain.invoke(
    {
        "question": "MoE 모델의 EN-FR BLEU 스코어는 무엇입니까?"
    }
)